<a href="https://colab.research.google.com/github/Hrishikesh-Harsh/Text_Classification_IR/blob/main/Text_Classifier_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use TF-IDF as prescribed
### Treat Abstract, Key and Title differently
### Order: $Key > Abstract ≈ Title$
### Hence, $W_k > W_a \approx W_t$
### For TF, we can use $TF = 1 + log(n_t*W_t + n_k*W_k + n_a*W_a)$
### One More Hyper-parameter to vary is Window size for (Word,Word) pairs
### Use $3$ different Window sizes for $Key, Abstract, Title$: $Win_k, Win_a, Win_t$

In [ ]:
!git clone https://github.com/yao8839836/text_gcn.git

Cloning into 'text_gcn'...
remote: Enumerating objects: 26801, done.
remote: Total 26801 (delta 0), reused 0 (delta 0), pack-reused 26801
Receiving objects: 100% (26801/26801), 861.51 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (205/205), done.
Updating files: 100% (26397/26397), done.


In [ ]:
cd text_gcn

[Errno 2] No such file or directory: 'text_gcn'
/content


In [ ]:
!pip3 install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 remove_words.py 20ng

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{"shan't", 'aren', 'to', 'then', 'other', 'herself', 'ain', 'theirs', 'just', 'below', "needn't", 'why', 'on', 'haven', 'too', 'there', 'should', 'her', 'those', 've', 'no', 'nor', 'so', 'shan', 'what', 'and', 'when', 'once', 'me', 'themselves', 'out', 'itself', 'will', "it's", 'very', 'about', 'at', 'y', 'my', 'mustn', 'most', "shouldn't", 'hadn', 'who', "didn't", 'both', 'd', 'isn', 'where', 'is', 'against', "won't", 'i', 'here', 'myself', 'now', 'been', 'before', 'can', "you'd", 'as', 'in', 'we', "hadn't", 'while', 'do', 'not', 'into', 'but', 'm', 'did', 'same', 'ours', "you're", 'doesn', 'them', 'our', 'of', "she's", 'wasn', 'further', "aren't", 'you', "weren't", 'his', 'being', 'have', 'didn', 'they', 'their', 's', 'yourselves', "you've", 'it', 'each', 'during', 'having', 'all', 'its', 'be', 'your', 'she', 'any', "mightn't", 'he', 'above', "wouldn't", "you'll", 'down', 'a

In [ ]:
!python build_graph.py 20ng

[7532, 7533, 7534, 7535, 7536, 7537, 7538, 7539, 7540, 7541, 7542, 7543, 7544, 7545, 7546, 7547, 7548, 7549, 7550, 7551, 7552, 7553, 7554, 7555, 7556, 7557, 7558, 7559, 7560, 7561, 7562, 7563, 7564, 7565, 7566, 7567, 7568, 7569, 7570, 7571, 7572, 7573, 7574, 7575, 7576, 7577, 7578, 7579, 7580, 7581, 7582, 7583, 7584, 7585, 7586, 7587, 7588, 7589, 7590, 7591, 7592, 7593, 7594, 7595, 7596, 7597, 7598, 7599, 7600, 7601, 7602, 7603, 7604, 7605, 7606, 7607, 7608, 7609, 7610, 7611, 7612, 7613, 7614, 7615, 7616, 7617, 7618, 7619, 7620, 7621, 7622, 7623, 7624, 7625, 7626, 7627, 7628, 7629, 7630, 7631, 7632, 7633, 7634, 7635, 7636, 7637, 7638, 7639, 7640, 7641, 7642, 7643, 7644, 7645, 7646, 7647, 7648, 7649, 7650, 7651, 7652, 7653, 7654, 7655, 7656, 7657, 7658, 7659, 7660, 7661, 7662, 7663, 7664, 7665, 7666, 7667, 7668, 7669, 7670, 7671, 7672, 7673, 7674, 7675, 7676, 7677, 7678, 7679, 7680, 7681, 7682, 7683, 7684, 7685, 7686, 7687, 7688, 7689, 7690, 7691, 7692, 7693, 7694, 7695, 7696, 7697, 769

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import csv
import math
import nltk
import re
import numpy as np
import scipy.sparse as sp
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import tensorflow as tf

In [21]:
# Loading original .csv file

file = open('/content/drive/MyDrive/IR_Project/dataset/PubMed.csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

In [22]:
# Set of stopwords to be removed
stop_words = set(stopwords.words('english'))

In [ ]:
train_ratio = 0.7
test_ratio = 0.3
N = 207

train_N = int(train_ratio*N)
test_N = int(test_ratio*N)

In [23]:
# Just the first line of the .csv file (Column Names)
header = []
header = next(csvreader)

In [24]:
# vocab_words would be a set of all distinct words found in the dataset/.csv file minus the stopwords
vocab_words = set()
vocab_words_list = []

In [25]:
rows = []
index = 0
title = []
keyword = []
abstract = []
for r in csvreader: 
  r_sub = []
  r[2] = r[2].replace('.',' ')
  r[2] = r[2].replace(',',' ')
  r[2] = r[2].replace(';',' ')
  r[2] = r[2].replace('|',' ')
  r[2] = r[2].replace('<',' ')
  r[2] = r[2].replace('>',' ')
  r[2] = r[2].replace(':',' ')
  r[2] = r[2].replace('=',' ')
  r[2] = r[2].replace('(',' ')
  r[2] = r[2].replace(')',' ')
  r[2] = r[2].replace('[',' ')
  r[2] = r[2].replace(']',' ')
  r[2] = r[2].replace('?',' ')


  r[3] = r[3].replace('.',' ')
  r[3] = r[3].replace(',',' ')
  r[3] = r[3].replace(';',' ')
  r[3] = r[3].replace('|',' ')
  r[3] = r[3].replace('<',' ')
  r[3] = r[3].replace('>',' ')
  r[3] = r[3].replace(':',' ')
  r[3] = r[3].replace('=',' ')
  r[3] = r[3].replace('(',' ')
  r[3] = r[3].replace(')',' ')
  r[3] = r[3].replace('[',' ')
  r[3] = r[3].replace(']',' ')
  r[3] = r[3].replace('?',' ')

  r[4] = r[4].replace('.',' ')
  r[4] = r[4].replace(',',' ')
  r[4] = r[4].replace(';',' ')
  r[4] = r[4].replace('|',' ')
  r[4] = r[4].replace('<',' ')
  r[4] = r[4].replace('>',' ')
  r[4] = r[4].replace(':',' ')
  r[4] = r[4].replace('=',' ')
  r[4] = r[4].replace('(',' ')
  r[4] = r[4].replace(')',' ')
  r[4] = r[4].replace('[',' ')
  r[4] = r[4].replace(']',' ')
  r[4] = r[4].replace('?',' ')

  words_title = word_tokenize(r[2])     # Tokenize the Title of that doc (row)
  words_keyword = word_tokenize(r[3])   # Tokenize the Keywords of that doc (row)
  words_abstract = word_tokenize(r[4])  # Tokenize the Abstract of that doc (row)

  title.append([])
  keyword.append([])
  abstract.append([])

  w_t = ""
  w_k = ""
  w_a = ""

  ''' 
      - Adding all non-stop-words to the vocabulary
      - Also maintaining doc-wise collection of Keywords, Title and Abstract words 
  '''

  for w in words_title:
    if w not in stop_words and w!="'s":
        w_t = w_t + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        title[index].append(w)
  
  for w in words_keyword:
    if w not in stop_words and w!="'s":
        w_k = w_k + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        keyword[index].append(w)

  for w in words_abstract:
    if w not in stop_words and w!="'s":
        w_a = w_a + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        abstract[index].append(w)
  
  index=index+1

  '''
    - Creating 'rows' to write back to Clean File
  '''
  for i in range(0,len(r)):
    if(i==2):
      r_sub.append(w_t)
    elif(i==3):
      r_sub.append(w_k)
    elif(i==4):
      r_sub.append(w_a) 
    else:
      r_sub.append(r[i])
    
  rows.append(r_sub)


In [ ]:
train_docs = []
test_docs = []

train_docs = rows[:train_N]
test_docs = rows[train_N:]

In [26]:
for wd in vocab_words:
  vocab_words_list.append(wd)

In [27]:
# Globals
docs_size = len(rows)
vocab_size = len(vocab_words)
wt_k = 3    # Weight to be given to Keywords in tf score
wt_t = 1    # Weight to be given to Title in tf score
wt_a = 1    # Weight to be given to Abstract in tf score
window_size = 20 # Window size for PMI calculation

In [ ]:
'''
  To write back to Clean File
'''
file = open('/content/drive/MyDrive/IR_Project/dataset/PubMed_Clean.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)

for r in rows:
  writer.writerow(r)


In [28]:
'''
  Create a dict to store {word: (ID, idf)} mapping
'''
dict_vocab = {}

for i,w in enumerate(vocab_words):
  dict_vocab[w]=(i,0)

In [29]:
for i,w in enumerate(vocab_words):
  if(i>5):
    break
  print(w)
print(vocab_size)

OSA
Semi-automated
3-4
Image
mortalities
Bradycardia
59464


In [ ]:
file = open('/content/drive/MyDrive/IR_Project/dataset/idf.csv', 'w', newline='') 
writer = csv.writer(file) 
writer.writerow(["word","idf"])
for wd in dict_vocab: 
  count = 0
  for i,doc in enumerate(rows):
    flag = 0 
    for w_t in title[i]:
      if(wd==w_t):
        count=count+1
        flag = 1
        break 

    if(flag==1):
      continue 

    for w_k in keyword[i]:
      if(wd==w_k):
        count=count+1
        flag = 1
        break

    if(flag==1):
      continue 

    for w_a in abstract[i]:
      if(wd==w_a):
        count=count+1
        flag = 1
        break

    if(flag==1):
      continue

  (id,idf) = dict_vocab[wd]
  if(count==0):
    idf = 0
  else:
    idf = math.log((docs_size/count),10) 
  dict_vocab[wd] = (id,idf)
  templine=[]
  templine.append(wd)
  templine.append(idf)
  writer.writerow(templine)
  print(id,templine)
  count = 0
writer.writerow("fine")
file.close()

Streaming output truncated to the last 5000 lines.
54464 ['dilatational', 3.9461082304369057]
54465 ['MFR', 3.9461082304369057]
54466 ['237', 3.344048239108943]
54467 ['Treat', 3.6450782347729245]
54468 ['mg/mL', 3.468986975717243]
54469 ['sufficient/adequate', 3.9461082304369057]
54470 ['CRISPR/Cas9-edited', 3.9461082304369057]
54471 ['psychosis-predisposing', 3.9461082304369057]
54472 ['gastroenterologic', 3.9461082304369057]
54473 ['odette', 3.9461082304369057]
54474 ['worldwide', 1.2842955448996447]
54475 ['approach', 1.199474031499327]
54476 ['heptapeptide', 3.9461082304369057]
54477 ['mid-August', 3.9461082304369057]
54478 ['soluble', 2.6036855496146996]
54479 ['kindergarten', 3.6450782347729245]
54480 ['WEMWBS', 3.468986975717243]
54481 ['near-complete', 3.6450782347729245]
54482 ['VA', 2.991865720997581]
54483 ['practitioner', 2.6450782347729245]
54484 ['roaming', 3.9461082304369057]
54485 ['Microminerals', 3.9461082304369057]
54486 ['preventionists', 3.9461082304369057]
54487 

In [30]:
file = open('/content/drive/MyDrive/IR_Project/dataset/idf.csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

header = []
header = next(csvreader)

In [31]:
for val in dict_vocab: 
  r = next(csvreader)
  if(r[0] not in dict_vocab):
    continue
  if(r[1]=='i'):
    break
  (id,idf) = dict_vocab[r[0]]
  idf = float(r[1])
  dict_vocab[r[0]] = (id,idf)

In [32]:
for i,ele in enumerate(dict_vocab):
  if(i==5):
    break;
  
  print(ele,",",dict_vocab[ele])

OSA , (0, 3.468986975717243)
Semi-automated , (1, 3.9461082304369057)
3-4 , (2, 3.101010190422649)
Image , (3, 2.7156593090586316)
mortalities , (4, 3.043018243444962)


In [33]:
Adj_Matrix = np.zeros((docs_size+vocab_size,docs_size+vocab_size))
weights = []
row_list = []
col_list = []

In [34]:
for i,doc in enumerate(rows):
  # for w_k in keyword[i]:
  #   Adj_Matrix[i,dict_vocab[w_k][0]+docs_size] += wt_k
  #   Adj_Matrix[dict_vocab[w_k][0]+docs_size,i] += wt_k

  # for w_t in title[i]:
  #   Adj_Matrix[i,dict_vocab[w_t][0]+docs_size] += wt_t
  #   Adj_Matrix[dict_vocab[w_t][0]+docs_size,i] += wt_t

  # for w_a in abstract[i]:
  #   Adj_Matrix[i,dict_vocab[w_a][0]+docs_size] += wt_a
  #   Adj_Matrix[dict_vocab[w_a][0]+docs_size,i] += wt_a
  wt = 0
  for w_k in keyword[i]:
    wt += wt_k
    weights.append(wt*dict_vocab[w_k][1])
    row_list.append(i)
    col_list.append(dict_vocab[w_k][0]+docs_size)
  for w_t in title[i]:
    wt += wt_t
    weights.append(wt*dict_vocab[w_t][1])
    row_list.append(i)
    col_list.append(dict_vocab[w_t][0]+docs_size)
  for w_a in abstract[i]:
    wt += wt_a
    weights.append(wt*dict_vocab[w_a][1])
    row_list.append(i)
    col_list.append(dict_vocab[w_a][0]+docs_size)


[65712, 54753, 45545, 61224, 48352]

In [ ]:
adj_header = []
for i in range(0,Adj_Matrix.shape[0]):
  if(i>=docs_size):
    adj_header.append("Word-"+str(i-docs_size))
  else:
    adj_header.append("Doc-"+str(i))

In [ ]:
file = open('/content/drive/MyDrive/IR_Project/dataset/Adj_M.csv', 'w', newline='') 
writer = csv.writer(file) 
writer.writerow(adj_header)

for i in range(0,Adj_Matrix.shape[0]):
  row_i = []
  for j in range(0,Adj_Matrix.shape[0]):
    if(Adj_Matrix[i,j]!=0):
      if(i<j):
        wd = vocab_words_list[j-docs_size]
        idf = dict_vocab[wd][1]
        Adj_Matrix[i,j] = (1+math.log(Adj_Matrix[i,j],10))*idf
        print(i,",",j)
        # print(wd,"|",idf,"|",Adj_Matrix[i,j])
      else:
        wd = vocab_words_list[i-docs_size]
        idf = dict_vocab[wd][1]
        Adj_Matrix[i,j] = (1+math.log(Adj_Matrix[i,j],10))*idf
        print(i,",",j)
        # print(wd,"|",idf,"|",Adj_Matrix[i,j])
    
    row_i.append(Adj_Matrix[i,j])
  
  writer.writerow(row_i)
file.close()

Streaming output truncated to the last 5000 lines.
67799 , 5109
67799 , 5294
67799 , 5341
67799 , 5427
67799 , 5539
67799 , 5541
67799 , 5548
67799 , 5593
67799 , 5744
67799 , 5788
67799 , 5789
67799 , 6017
67799 , 6020
67799 , 6028
67799 , 6092
67799 , 6101
67799 , 6116
67799 , 6136
67799 , 6142
67799 , 6154
67799 , 6261
67799 , 6345
67799 , 6522
67799 , 6866
67799 , 6935
67799 , 6981
67799 , 7146
67799 , 7214
67799 , 7218
67799 , 7222
67799 , 7223
67799 , 7225
67799 , 7227
67799 , 7262
67799 , 7396
67799 , 7419
67799 , 7774
67799 , 7795
67799 , 7819
67799 , 7841
67799 , 7878
67799 , 8328
67799 , 8357
67799 , 8408
67799 , 8471
67799 , 8516
67799 , 8606
67799 , 8678
67799 , 8700
67799 , 8719
67799 , 8728
67799 , 8787
67800 , 850
67801 , 2019
67801 , 4981
67802 , 11
67802 , 123
67802 , 144
67802 , 254
67802 , 290
67802 , 292
67802 , 304
67802 , 322
67802 , 352
67802 , 371
67802 , 407
67802 , 441
67802 , 449
67802 , 472
67802 , 601
67802 , 640
67802 , 654
67802 , 658
67802 , 672
67802 , 

In [36]:
# word co-occurence with context windows
windows = []

for row in rows:
    content=row[2]+" "+row[3]+" "+row[4]
    words = content.split()
    length = len(words)
    if length <= window_size: 
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)

print(len(windows))

#calculating p(i) , word_window_freq has the number of windows a particular word appears in across all windows.
word_window_freq = {}
k = 0
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])
    print("k=",k)  
    k+=1


word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i] #ith word in window
            word_i_id = dict_vocab[word_i][0]
            word_j = window[j] #jth word in range 0-i in the same window
            word_j_id = dict_vocab[word_j][0]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id) #concat id and use it to count a pair or p(i,j)
            if word_pair_str in word_pair_count: #word_pair_count stores number of pairs along with number of times they appear.
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
        print("i=",i)
        


# row = []
# col = []
# weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    print("pair(i,j)=",i,j)
    count = word_pair_count[key] #p(i,j)
    word_freq_i = word_window_freq[vocab_words_list[i]] #p(i)
    word_freq_j = word_window_freq[vocab_words_list[j]] #p(j)
    pmi = math.log((1.0 * count / num_window) /(1.0 * word_freq_i * word_freq_j/(num_window * num_window)),10) #adj(i,j)
    if pmi <= 0:
        continue
    # Adj_Matrix[docs_size+i][docs_size+j]=pmi
    weights.append(pmi)
    row_list.append(docs_size+i)
    col_list.append(docs_size+j)


Streaming output truncated to the last 5000 lines.
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10
i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6

KeyboardInterrupt: ignored

In [ ]:
for i in range(Adj_Matrix.shape[0]):
      Adj_Matrix[i,i]=1